In [ ]:
import pandas as pd

# Мердж таблиц

Таблица со всей статистикой игроков.

In [ ]:
player_stats = pd.read_csv('/content/drive/MyDrive/xg_project/Players data 3.csv')
player_stats.replace(',','', regex=True, inplace=True) # В столбце с минутами тысячи отделены запятыми, удаляем их и кастим к флоату
player_stats['minutes'] = player_stats['minutes'].astype('float')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (195) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Функция возвращающая список таблиц с трансферами для каждого игрока


In [ ]:
def get_seasons(player, table, max_seasons, min_games):
  
  df = table[table['name'] == player].copy()
  df['year'] = df['season'].astype(str).str[:4]
  df['year'] = df['year'].astype('float64')
  is_keeper = (df.iloc[-1,:]['all_goals_against_gk'] == df.iloc[-1,:]['all_goals_against_gk'])
  if (is_keeper):
    df = df[df['psxg_gk'] == df['psxg_gk']]
  else:
    df = df[df['xg'] == df['xg']]

  df = df[df['comp_level'] != '1. MLS'] #убираем млс

  #убрали сезоны с неполной статистикой
  if (len(df['squad'].unique()) < 2):
    print("Ошибка: Только одна команда со статистикой для " + player)
    return ([], is_keeper)
  
  # В начало таблицы могло попасть начало сезона в другой команде, по которой
  # мало статистики, например Валенсия у Канселу, избавимся от этого
  # НАДО ЗАМЕНИТЬ НА ОГРАНИЧЕНИЕ МИНИМАЛЬНОГО ЧИСЛА ИГР
  # if (df.iloc[0,:]['age'] == df.iloc[1,:]['age']):
    # df = df.iloc[1:,:]

  # составим список всех трансферов
  cur_team = df.iloc[0,:]['squad']
  transfers = []
  for i in range(1, len(df.index)):
    new_team = df.iloc[i,:]['squad']
    if (cur_team != new_team):
      if (df.iloc[i,:]['Unnamed: 0'] - 1 == df.iloc[i - 1,:]['Unnamed: 0'] and df.iloc[i,:]['games'] >= min_games):
        transfers.append(i)
      cur_team = new_team

  season_tables = [] #список с датафреймами для каждого трансфера игрока
  for i in transfers:
    old_team = df.iloc[i - 1,:]['squad']
    old_age = df.iloc[i - 1,:]['age']
    new_team = df.iloc[i,:]['squad']
    new_age = df.iloc[i,:]['age']
    l = i - 1
    while (l > 0):
      if (old_team != df.iloc[l - 1,:]['squad'] or old_age - 1 != df.iloc[l - 1,:]['age']):
        break
      l -= 1
    r = i
    while (r < len(df.index) - 1):
      if (new_team != df.iloc[r + 1,:]['squad'] or new_age + 1 != df.iloc[r + 1,:]['age']):
        break
      r += 1
    #обрежем, если взяли больше сезонов чем нужно
    l = max(l, i - max_seasons)
    r = min(r, i + max_seasons - 1)
    season_tables.append(df.iloc[l:r + 1,:])
  return (season_tables, is_keeper)
 

Строим списки датафреймов для каждого игрока, ограничиваем 1 годом до и после трансфера. Требуем минимум 10 игр после трансфера.

In [ ]:
players = list(player_stats.name.unique())
field = []
keepers = []
for player in players:
    transfers, is_keeper = get_seasons(player, player_stats, 1, 10)
    if is_keeper:
        for df in transfers:
            keepers.append(df)
    else:
        for df in transfers:
            field.append(df)

Загрузим словарь соответствий имен игроков из таблиц fbref и transfermarkt

In [ ]:
names = pd.read_csv('/content/drive/MyDrive/xg_project/dict_names.csv')
tmp = names.to_dict()
dict_names = {}
for key, value in tmp.items():
  dict_names[key] = value[0]

По каким то причинам в словаре отсутствует Педро Порро, хотя он есть в обоих таблицах, добавим его вручную.

In [ ]:
dict_names['Pedro Antonio Porro Sauceda'] = 'Pedro Porro'

Заменим имена игроков из таблицы с fbref на имена с трансфермакта:

In [ ]:
from tqdm.notebook import tqdm

for df in tqdm(field):
  df.replace({'name' : dict_names}, inplace= True)

Загрузим таблицу с данными о трансферах с трансфермаркта:

In [ ]:
transfers = pd.read_csv('/content/drive/MyDrive/xg_project/Transfers.csv')

Что бы не заморачиваться с мерджем просто составим словарь соответствий имен и позиций

In [ ]:
positions = {}
for i, row in transfers.iterrows():
  positions[row.player_name] = row.position

In [ ]:
count = 0
for df in tqdm(field):
  position = positions[df.iloc[0]['name']]
  df['position'] = position

Импортируем статистику клубов

In [ ]:
team_stats1 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats1.csv')
team_stats2 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats2.csv')
team_stats3 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats3.csv')
team_stats4 = pd.read_csv('/content/drive/MyDrive/xg_project/team_stats4.csv')
team_stats = pd.concat([team_stats1, team_stats2, team_stats3, team_stats4])

In [ ]:
team_stats

Функция для получения названий команд до и после трансфера, года последнего сезона в старой команде и окна трансфера. Летний трансфер может считатьс за зимний, если игрок успел провести несколько матчей за старую команду до трансфера.

In [ ]:
def get_info(table):
  # возвращает (название 1 команды, название 2 команды, год начала статистики, последний год в первой команде, окно трансфера (1 если зимний))
  y = table.iloc[0]['year']
  y0 = y
  team1 = table.iloc[0]['squad']
  i = 0;
  while (table.iloc[i]['squad'] == team1):
    y = table.iloc[i]['year']
    i += 1
  team2 = table.iloc[i]['squad']
  winter = (y == table.iloc[i]['year'])
  return (team1, team2, y0, y, winter)

In [ ]:
print(get_info(field[701]))

('Atalanta', 'Milan', 2019.0, 2019.0, True)


Функция для получения нужных таблиц с командной статистикой

In [ ]:
def get_tables(table):
  t1, t2, y0, y, is_win = get_info(table)
  df1 = team_stats[team_stats['team'] == t1]
  df1 = df1[(df1['year'] >= y0) & (df1['year'] <= y)]
  df2 = team_stats[team_stats['team'] == t2]
  df2 = df2[(df2['year'] >= y0) & (df2['year'] <= y)]
  return [df1, df2]

Функция Акима, которая мерджит статистику игрока и команд для каждого трансфера

In [ ]:
from tqdm.notebook import tqdm

merge_table = pd.DataFrame()
for i, observation in enumerate(tqdm(field)):

    # Check there are no missing seasons
    if observation.year.values[1] - observation.year.values[0] > 1:
        continue

    old, new = get_tables(observation)
    old.columns = [x + '_old' for x in old.columns]
    new.columns = [x + '_new' for x in new.columns]

    old.reset_index(drop=True, inplace=True)
    new.reset_index(drop=True, inplace=True)

    club_features = pd.concat([old, new], axis=1)
    # club_features = pd.DataFrame(club_features.mean()).T
    club_features['year'] = club_features['year_old']
    club_features['predict_year'] = club_features['year_old']

    if get_info(observation)[-1] != 1:
        observation['predict_year'] = observation['year'] - 1
        merge_before_target = club_features.merge(observation.drop(columns='predict_year'), on=['year'], how='left')
        merge = merge_before_target.merge(observation.drop(columns='year'), on=['predict_year'], how='left')
    else:
      merge_before_target = club_features.merge(observation.iloc[:1], on=['year'], how='left')
      merge = merge_before_target.merge(observation.iloc[1:], on=['year'], how='left')

    merge_table = merge_table.append(merge)

    if merge['sca_per90_y'].isnull().sum() > 0:
      print(i, observation[['name', 'squad', 'year']])

# merge_table.drop(columns = ['year_old', 'year_new', 'predict_year'], inplace=True)

merge_table.head()

,team_old,year_old,result_old,goals_for_old,goals_against_old,xg_for_old,xg_against_old,possession_old,shots_on_target_against_old,goals_against_gk_old,saves_old,save_pct_old,clean_sheets_old,psxg_gk_old,psxg_net_gk_old,pens_att_gk_old,pens_allowed_old,pens_saved_old,pens_missed_gk_old,passes_completed_launched_gk_old,passes_launched_gk_old,passes_pct_launched_gk_old,passes_gk_old,passes_throws_gk_old,pct_passes_launched_gk_old,passes_length_avg_gk_old,goal_kicks_old,pct_goal_kicks_launched_old,goal_kick_length_avg_old,crosses_gk_old,crosses_stopped_gk_old,crosses_stopped_pct_gk_old,def_actions_outside_pen_area_gk_old,avg_distance_def_actions_gk_old,goals_old,shots_total_old,shots_on_target_old,shots_on_target_pct_old,goals_per_shot_old,goals_per_shot_on_target_old,...,losses_gk_y,clean_sheets_y,clean_sheets_pct_y,pens_att_gk_y,pens_allowed_y,pens_saved_y,pens_missed_gk_y,pens_save_pct_y,free_kick_goals_against_gk_y,corner_kick_goals_against_gk_y,own_goals_against_gk_y,psxg_gk_y,psnpxg_per_shot_on_target_against_y,psxg_net_gk_y,psxg_net_per90_gk_y,passes_completed_launched_gk_y,passes_launched_gk_y,passes_pct_launched_gk_y,passes_gk_y,passes_throws_gk_y,pct_passes_launched_gk_y,passes_length_avg_gk_y,goal_kicks_y,pct_goal_kicks_launched_y,goal_kick_length_avg_y,crosses_gk_y,crosses_stopped_gk_y,crosses_stopped_pct_gk_y,def_actions_outside_pen_area_gk_y,def_actions_outside_pen_area_per90_gk_y,avg_distance_def_actions_gk_y,dom_lg_goals_against_gk_y,dom_lg_clean_sheets_y,dom_cup_goals_against_gk_y,dom_cup_clean_sheets_y,intl_cup_goals_against_gk_y,intl_cup_clean_sheets_y,all_goals_against_gk_y,all_clean_sheets_y,position_y
0,Huddersfield,2018,0.421053,0.578947,2.000000,0.760526,1.671053,47.026316,4.657895,2.000000,2.842105,58.763889,0.157895,1.655263,-0.318421,0.210526,0.184211,0.026316,0.000000,9.763158,23.236842,42.263158,28.000000,2.789474,63.926316,47.468421,6.947368,76.984211,58.386842,7.421053,0.842105,10.875676,0.605263,14.326316,0.526316,10.526316,3.105263,29.728947,0.043421,0.173784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Attacking Midfield
0,Saint-Étienne,2019,1.000000,1.029412,1.558824,0.985294,1.158824,49.205882,3.911765,1.558824,2.500000,58.260606,0.176471,1.402941,-0.097059,0.176471,0.147059,0.000000,0.029412,6.029412,15.764706,41.912121,22.235294,3.794118,47.944118,39.379412,7.617647,61.405882,46.767647,6.823529,0.294118,4.635294,0.411765,13.817647,0.970588,10.882353,3.294118,31.411765,0.078235,0.233939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Right Winger
0,Köln,2017,0.700000,1.050000,1.950000,1.244118,2.000000,47.875000,5.625000,1.950000,3.875000,70.137500,0.150000,2.100000,0.041176,0.225000,0.200000,0.025000,0.000000,8.617647,22.205882,37.164706,22.764706,2.411765,67.044118,49.261765,8.205882,86.376471,61.352941,10.882353,0.823529,8.735294,0.970588,15.794118,1.000000,11.925000,3.750000,31.462500,0.081250,0.293590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Centre-Back
0,Huesca,2018,0.868421,1.131579,1.710526,1.210526,1.500000,44.684211,4.394737,1.710526,2.947368,64.672973,0.157895,1.536842,-0.094737,0.236842,0.236842,0.000000,0.000000,8.815789,22.684211,38.797368,20.368421,2.000000,73.744737,52.931579,8.052632,91.155263,64.594737,9.157895,0.500000,6.007895,0.342105,12.913158,1.131579,12.368421,3.684211,30.436842,0.085789,0.260278,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Central Midfield
0,Sampdoria,2018,1.394737,1.578947,1.342105,1.310526,1.392105,55.105263,3.789474,1.368421,2.631579,66.968571,0.342105,1.355263,0.039474,0.210526,0.184211,0.026316,

У некоторых трансферов нет статистики у новой команды:

In [ ]:
df = merge_table.loc[:, columns].dropna(subset=['possession_new'])

In [ ]:
merge_table.shape, df.shape

((904, 834), (788, 16))

# Модель Акима

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
club_features   = ['possession', 'npxg', 'xg_against', 'passes_pct', 'touches']
player_features = ['minutes_90s', 'goals_per90', 'assists_per90', 'sca_per90', 'gca_per90']

In [ ]:
columns = [x + '_old' for x in club_features] + [x + '_new' for x in club_features] + [x + '_x' for x in player_features] + ['sca_per90_y']

In [ ]:
[x for x in old.columns if 'xg' in x]

['xg_for_old',
 'xg_against_old',
 'psxg_gk_old',
 'psxg_net_gk_old',
 'xg_old',
 'npxg_old',
 'npxg_per_shot_old',
 'xg_net_old',
 'npxg_net_old']

In [ ]:
df = merge_table.loc[:, columns].dropna(subset=['sca_per90_y', 'possession_new'])

In [ ]:
train, valid_plus_test = train_test_split(df, random_state=2021, test_size=0.3)
val, test = train_test_split(valid_plus_test, random_state=2021, test_size=0.67)

In [ ]:
val

,possession_old,npxg_old,xg_against_old,passes_pct_old,touches_old,possession_new,npxg_new,xg_against_new,passes_pct_new,touches_new,minutes_90s_x,goals_per90_x,assists_per90_x,sca_per90_x,gca_per90_x,sca_per90_y
0,61.553191,1.594737,0.660526,85.750000,867.815789,53.717391,1.407895,0.857895,81.463158,700.710526,29.4,0.07,0.10,3.21,0.41,1.71
0,52.578947,1.357895,1.118421,78.086842,636.763158,44.368421,0.802632,1.436842,73.681579,560.973684,1.5,0.00,0.00,2.67,0.00,0.20
0,43.500000,0.831579,1.563158,75.142105,546.973684,43.289474,1.055263,1.434211,72.239474,518.578947,19.0,0.00,0.00,1.27,0.05,1.83
0,50.413043,1.313043,1.132609,79.943478,629.521739,43.289474,1.055263,1.434211,72.239474,518.578947,8.6,0.00,0.12,2.32,0.35,2.73
0,43.921053,1.034211,1.089474,71.463158,538.342105,54.957447,1.626316,1.044737,82.555263,654.657895,33.6,0.09,0.09,3.24,0.18,2.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,54.344828,1.068966,1.379310,84.227586,689.896552,44.478261,0.847826,2.043478,75.804348,555.913043,0.0,0.00,0.00,0.00,0.00,1.35
0,51.750000,1.007143,1.346429,79.342857,640.678571,42.523810,0.938095,1.395238,76.052381,563.857143,2.6,0.00,0.00,1.55,0.00,2.07
0,47.447368,1.002632,1.218421,70.013158,531.184211,52.894737,1.281579,1.184211,81.639474,637.131579,33.1,0.03,0.15,2.30,0.30,2.37
0,44.210526,0.760526,1.381579,71.268421,507.973684,47.105263,1.063158,1.255263,72.705263,546.631579,34.7,0.03,0.00,2.33,0.09,1.54


In [ ]:
target_column = 'sca_per90_y'
lgb_train = lgb.Dataset(train.drop(columns=target_column).values, train[target_column].values)
lgb_val = lgb.Dataset(val.drop(columns=target_column).values, val[target_column].values)


params = {
    "objective" : "mse",
    "metric" : "mae",
    "max_depth" : 7,
    "num_leaves" : 100,
    "learning_rate" : 0.05,
    "bagging_fraction" : 0.3,
    "feature_fraction" : 0.15,
    "lambda_l1" : 5,
    "lambda_l2" : 5,
    "bagging_seed" : 42,
    "verbosity" : 1,
    "seed": 42
}

clf = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_val],
    num_boost_round=500,
    early_stopping_rounds=10,
    verbose_eval=10
)

Training until validation scores don't improve for 10 rounds.
[10]	valid_0's l1: 0.813454
[20]	valid_0's l1: 0.776109
[30]	valid_0's l1: 0.765316
[40]	valid_0's l1: 0.733746
[50]	valid_0's l1: 0.713998
[60]	valid_0's l1: 0.709616
[70]	valid_0's l1: 0.701773
[80]	valid_0's l1: 0.691587
[90]	valid_0's l1: 0.680149
[100]	valid_0's l1: 0.673268
[110]	valid_0's l1: 0.671308
Early stopping, best iteration is:
[107]	valid_0's l1: 0.669813


# Сплит

Пока есть небольшие проблемы с матчингом позиций и появилось несколько ошибочных вратарей.

Разделим общую таблицу на 5 частей для каждой группы позиций:

In [ ]:
dict_pos = {'Attacking Midfield' : 'Wingers', 'Right Winger' : 'Wingers', 'Centre-Back' : 'Centrebacks',
       'Central Midfield' : 'Midfielders', 'Left Midfield' : 'Wingers', 'Left-Back' : 'Fullbacks', 'Right Midfield' : 'Wingers',
       'Left Winger' : 'Wingers', 'Right-Back' : 'Fullbacks', 'Centre-Forward' : 'Forwards',
       'Defensive Midfield' : 'Midfielders', 'Second Striker' : 'Forwards', 'midfield' : 'Midfielders',
       'Goalkeeper' : 'Goalkeepers'} 
tables = [t for _, t in merge_table.replace({'position_y' : dict_pos}).groupby(['position_y'])]

In [ ]:
Centrebacks = tables[0]
Forwards = tables[1]
Fullbacks = tables[2]
Midfielders = tables[4]
Wingers = tables[5]

Функция для разбиения на трейн, тест и валидацию и их экспорт в csv

In [ ]:
def ttv(df, name):
  train, valid_plus_test = train_test_split(df, random_state=2021, test_size=0.3)
  val, test = train_test_split(valid_plus_test, random_state=2021, test_size=0.67)
  train.to_csv('/content/drive/MyDrive/xg_project/Train Test Val/' + name + '_train.csv')
  val.to_csv('/content/drive/MyDrive/xg_project/Train Test Val/' + name + '_val.csv')
  test.to_csv('/content/drive/MyDrive/xg_project/Train Test Val/' + name + '_test.csv')

In [ ]:
ttv(Centrebacks, 'Centerbacks')
ttv(Forwards, 'Forwards')
ttv(Fullbacks, 'Fullbacks')
ttv(Midfielders, 'Midfielders')
ttv(Wingers, 'Wingers')

# Функции для корректировки атрибутов на время и владение

Функция для корректировки стат на время

In [ ]:
def time_adj(df, list_of_attr):
  for attr in list_of_attr:
    df[attr] = (df[attr] * 90) / df['minutes']

Функция для корректировки стат на владение (считается что уже скорректировано на время) 

Если игрок провел в команде только половину сезона, то владение все равно берется по всему сезону

In [ ]:
def poss_adj(df, list_of_attr, on_ball): 
  # list_of_attr - список столбцов которые нужно скорректировать
  # on_ball - True для показателей на мяче (корректировка вниз при владении > 50%) и False без мяча (корр вверх)
  for i, row in df.iterrows():
    team = row['squad']
    year = row['year']
    dft = team_stats[team_stats['team'] == team]
    dft = dft[dft['year'] == year]
    dft = dft.iloc[0]
    if (dft['possession'] != dft['possession']):
      raise NotImplementedError #пока не решил что делать с комаднами без владения
    if on_ball:
      for attr in list_of_attr:
        df.loc[i, attr] = (row[attr] * 50) / dft['possession']
    else:
      for attr in list_of_attr:
        df.loc[i, attr] = (row[attr] * 50) / (100 - dft['possession'])